# Function Calling Tutorial PT 1

How does function calling (AKA tool calling) work?
1. You define your functions which can be api calls e.g. a weather function that connects to a weather api or could be a hardcoded function like a math function that takes 2 inputs and returns an output a+b
2. You put all the tools you wrote into the appropriate schema for OpenAI SDK - KEY: You must have your descriptions for both the tool and your input parameters on point.
3. INITIAL CALL TO THE LLM: The LLM will decided what function to use if any (can set the tool_choice to what you need) and the args (e.g. if you ask to find weather in sydney and you need lat and long for your function call the LLM will return the lat and long). Append this to messages.
4. You take those arguments and feed into your function to get your function output. Take the function output and append it to the messages.
5. SECOND CALL TO LLM: Now with your updated messages (includes initial user query, function call + args, function call output) you then feed back to LLM and get the final answer formatted nicely.

<p align="center">
    <img src="..\..\assets\function-calling\steps.png" alt="Function calling steps" width="300"/>
</p>
<p align="center">
    <img src="..\..\assets\function-calling\tool_choice.png" alt="Function calling steps" width="300"/>
</p>

In [15]:
from openai import OpenAI
import json
import os
from dotenv import load_dotenv
import requests

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [16]:
# Define the function you are going to use:
# Uses Open-Meteo's Weather Forecast API!!!
def get_weather(latitude, longitude):
    url = (
        f"https://api.open-meteo.com/v1/forecast?"
        f"latitude={latitude}&longitude={longitude}"
        f"&current=temperature_2m"
    )
    
    response = requests.get(url)
    data = response.json()
    
    return data.get('current').get('temperature_2m')

# example use
get_weather("-33.8727", "151.2057")

14.6

In [ ]:
# define list of callable tools for model
tools = [
    {
        "type": "function",
        "name": "get_current_weather",
        "description": "Get the current weather in degrees Celcius for a given latitude and longitude",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {
                    "type": "string",
                    "description": "The latitude of a place",
                },
                "longitude": {
                    "type": "string",
                    "description": "The longitude of a place",
                },
            },
            "required": ["latitude", "longitude"],
        }
    }
]

messages = [{"role": "user", "content": "What is the weather in Sydney?"}]

#for lesson 2 function calling: if we try the query below this code wouldn't work
messages = [{"role": "user", "content": "What is the weather in Sydney and Busan?"}]

response = client.responses.create(
    model="gpt-3.5-turbo",
    tools=tools,
    input=messages,
    tool_choice="auto",
)

#print(response.model_dump_json(indent=2))
print(response.output)

# WHAT HAPPENED?
# We called the api endpoint and it returned which tool we should use and the inputs

[ResponseFunctionToolCall(arguments='{"latitude":"-33.8688","longitude":"151.2093"}', call_id='call_REqSc4pbRu8lLhwl2OIlYd8T', name='get_current_weather', type='function_call', id='fc_08e03cabcfa9b58d0068c82c4c4cbc8190a87574c0d26d1cf9', status='completed')]


In [18]:
function_call = None
function_call_arguments = None
messages += response.output

for item in response.output:
    if item.type == "function_call":
        function_call = item
        function_call_arguments = json.loads(item.arguments)

print(function_call)
print(function_call_arguments)


ResponseFunctionToolCall(arguments='{"latitude":"-33.8688","longitude":"151.2093"}', call_id='call_REqSc4pbRu8lLhwl2OIlYd8T', name='get_current_weather', type='function_call', id='fc_08e03cabcfa9b58d0068c82c4c4cbc8190a87574c0d26d1cf9', status='completed')
{'latitude': '-33.8688', 'longitude': '151.2093'}


In [19]:
# 3. Execute the function logic for get_weather
result = {"weather": get_weather(function_call_arguments["latitude"], 
                                 function_call_arguments["longitude"])}
print(result)

{'weather': 14.2}


In [20]:
# 4. Provide function call results to the model
messages.append({
    "type": "function_call_output",
    "call_id": function_call.call_id,
    "output": json.dumps(result),
})

print("Final input:")
from pprint import pprint
pprint(messages)

Final input:
[{'content': 'What is the weather in Sydney?', 'role': 'user'},
 ResponseFunctionToolCall(arguments='{"latitude":"-33.8688","longitude":"151.2093"}', call_id='call_REqSc4pbRu8lLhwl2OIlYd8T', name='get_current_weather', type='function_call', id='fc_08e03cabcfa9b58d0068c82c4c4cbc8190a87574c0d26d1cf9', status='completed'),
 {'call_id': 'call_REqSc4pbRu8lLhwl2OIlYd8T',
  'output': '{"weather": 14.2}',
  'type': 'function_call_output'}]


In [21]:
response = client.responses.create(
    model="gpt-3.5-turbo",
    tools=tools,
    input=messages,
)

# 5. The model should be able to give a response!
print("Final output:")
print(response.output_text + "\n\n")
print(response.model_dump_json(indent=2))


Final output:
The current weather in Sydney is 14.2 degrees Celsius.


{
  "id": "resp_08e03cabcfa9b58d0068c82c4e9b0c8190b598ad05e6136281",
  "created_at": 1757949006.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-3.5-turbo-0125",
  "object": "response",
  "output": [
    {
      "id": "msg_08e03cabcfa9b58d0068c82c533dc48190b79f517dde032ce4",
      "content": [
        {
          "annotations": [],
          "text": "The current weather in Sydney is 14.2 degrees Celsius.",
          "type": "output_text",
          "logprobs": []
        }
      ],
      "role": "assistant",
      "status": "completed",
      "type": "message"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "tools": [
    {
      "name": "get_current_weather",
      "parameters": {
        "type": "object",
        "properties": {
          "latitude": {
            "type": "string",
            "description": "The 